# Capítulo 06 — Fine-Tuning: A Especialização do Modelo

Neste capítulo, vamos realizar uma "cirurgia neural". Pegaremos o GPTMini que aprendeu a ler e escrever no Capítulo 05 e o ensinaremos a classificar mensagens como **Normal** ou **Spam**.

--- 
### 🎯 O Poder da Especialização
O Fine-tuning não apaga o que o modelo sabe; ele apenas direciona esse conhecimento para uma tarefa específica. Substituiremos a "cabeça de vocabulário" por uma "cabeça de decisão".

![Pretrain vs Finetune](./infograficos/01-pretrain-vs-finetune.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"

if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

print("📂 Diretório atual:", os.getcwd())

In [ ]:
# ============================================================
# 1. Setup e Conexão com a Inteligência Base
# ============================================================
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 O arquivo 'gpt_checkpoint.pt' não foi encontrado localmente.")
    print("Por favor, suba o checkpoint gerado no final do Capítulo 05:")
    uploaded = files.upload()

try:
    ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
    stoi, itos = ckpt['stoi'], ckpt['itos']
    vocab_size = len(stoi)
    print(f"✅ Inteligência Base Carregada! Vocabulário: {vocab_size} caracteres.")
    print(f"🔹 Configuração original detectada: Layers={ckpt['config'].n_layers}, Heads={ckpt['config'].n_heads}")
except Exception as e:
    print(f"❌ ERRO AO CARREGAR: {e}")
    print("Certifique-se de que você salvou o checkpoint corretamente no Capítulo 05.")

## 2. Preparando os Dados de Missão

Precisamos de exemplos de SPAM para que o modelo entenda o padrão de mensagens maliciosas.

In [ ]:
raw_data = [
    ("ganhe 1 milhao agora clique aqui", 1), # Spam
    ("oferta imperdivel premio gratis", 1),   # Spam
    ("seu premio esta esperando resgate", 1), # Spam
    ("ola tudo bem como voce esta", 0),      # Normal
    ("reuniao de equipe amanha as dez", 0),   # Normal
    ("voce vai no churrasco no domingo", 0)   # Normal
]

encode = lambda s: [stoi[c] for c in s.lower() if c in stoi]

def build_dataset(data, max_len=32):
    X, Y = [], []
    for text, label in data:
        ids = encode(text)
        # Padding para garantir que todas as sequências tenham o mesmo tamanho
        ids = ids[:max_len] + [stoi.get(' ', 0)] * (max_len - len(ids))
        X.append(ids)
        Y.append(label)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device)

X_train, Y_train = build_dataset(raw_data)
print(f"📊 Dataset Processado: {len(X_train)} exemplos prontos para o treino.")

## 3. Criando o Classificador

Aqui, acoplamos a "Cabeça de Classificação" ao Backbone do Transformer.

![Classification Head](./infograficos/02-classification-head.png)

In [ ]:
class GPTClassifier(nn.Module):
    def __init__(self, backbone, num_classes=2):
        super().__init__()
        self.backbone = backbone
        # Camada que converte os neurônios do GPT em 2 opções (Normal/Spam)
        self.clf_head = nn.Linear(backbone.config.d_model, num_classes)
        
    def forward(self, x):
        x = self.backbone.emb(x)
        x = self.backbone.blocks(x)
        x = self.backbone.ln_f(x)
        # Usamos o último token para representar o significado da frase inteira (Pooling)
        last_token_feat = x[:, -1, :]
        return self.clf_head(last_token_feat)

# Inicializamos o Backbone com a inteligência do Cap 05
backbone = GPTMini(ckpt['config']).to(device)
backbone.load_state_dict(ckpt['state_dict'])

model = GPTClassifier(backbone).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
print("🏗️ Modelo especializado pronto para o treinamento.")

## 4. O Treinamento do Especialista

Damos 200 passos de ajuste fino. O modelo deve parar de chutar e começar a ter certeza.

In [ ]:
print("🚀 Iniciando Especialização...")
loss_history = []
model.train()

for step in range(201):
    logits = model(X_train)
    loss = F.cross_entropy(logits, Y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    loss_history.append(loss.item())
    if step % 50 == 0: 
        preds = torch.argmax(logits, dim=-1)
        acc = (preds == Y_train).float().mean()
        print(f"Passo {step:03d} | Erro: {loss.item():.4f} | Acurácia: {acc.item()*100:.1f}%")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#34A853')
plt.title("Curva de Aprendizado do Especialista")
plt.show()

## 5. Teste de Campo: Identificando Spams Reais

Vamos testar com frases inéditas para ver se ele generalizou o conceito de Spam.

In [ ]:
def classify(text):
    model.eval()
    with torch.no_grad():
        ids = encode(text)
        # Padding manual para 32 caracteres
        ids_tensor = torch.tensor(ids[:32] + [stoi.get(' ', 0)] * (32 - len(ids))).unsqueeze(0).to(device)
        
        logits = model(ids_tensor)
        probs = F.softmax(logits, dim=-1)
        pred = torch.argmax(probs, dim=-1).item()
        
        label = "🚨 SPAM" if pred == 1 else "✅ NORMAL"
        conf = probs[0, pred].item() * 100
        return f"{label} ({conf:.1f}% de confiança)"

print("🔍 TESTANDO O ESPECIALISTA:")
print("-" * 30)
frases = [
    "ganhe seu premio agora mesmo gratis",
    "oi amigo voce vai na aula hoje",
    "clique aqui para resgatar 1 milhao"
]

for f in frases:
    print(f"Frase: '{f}'")
    print(f"Resultado: {classify(f)}\n")